### Importowanie pakietów z znanym nam zbiorem iris

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

In [ ]:
iris = load_iris()
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [ ]:
type(iris)

In [ ]:
iris

In [ ]:
type(df)

In [ ]:
df.head()

In [ ]:
!pip install sqlalchemy

### Tworzenie i łączenie się z bazą danych SQL

Polecenie create_engine pozwala nam się podłączyć do już istniejącej bazy danych.
SQLite to baza danych "plikowa". Stworzy nam się plik .db, który będzie zawierałnasze bazy danych

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///irysy4.db')
baza_danych = create_engine('sqlite:///irysy5.db')

Za pomocą polecenia to_sql możemy zapisać nasze obiekty jako tabele sql-owe i zapisać w bazie danych

In [ ]:
df.to_sql('dane', con=engine, index=False)

In [ ]:
a=engine.execute("SELECT * FROM dane").fetchall()

In [ ]:
engine.execute("SELECT AVG([sepal width (cm)]) FROM dane").fetchall()

In [ ]:
type(a)

In [ ]:
a

In [ ]:
a[0]

In [ ]:
type(a[0])

Te dane możemy skonwertować do data frame

In [ ]:
pd.DataFrame(a)

In [ ]:
df2 = pd.DataFrame({'name' : ['User 1', 'User 2', 'User 3']})
df2

In [ ]:
df2.to_sql('users', con=engine, if_exists="replace",index=True)
#engine.execute("SELECT * FROM users").fetchall()

In [ ]:
df3 = pd.DataFrame({'name' : ['User 6', 'User 7']})
df3.to_sql('users', con=engine, if_exists='append')

In [ ]:
engine.execute("SELECT * FROM users").fetchall()

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import Column, String, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [ ]:
engine = create_engine('sqlite:///irysy2.db')
base = declarative_base()

In [ ]:
type(base)

In [ ]:
class Transakcje(base):
    
    __tablename__ = 'transakcje'
    
    transakcja_id = Column(Integer, primary_key=True)
    data = Column(String)
    przedmiot_id = Column(Integer)
    cena = Column(Integer)
    
    def __init__(self, transakcja_id, data, przedmiot_id, cena):
        self.transakcja_id = transakcja_id
        self.data = data
        self.przedmiot_id = przedmiot_id
        self.cena = cena

In [ ]:
class Kwiaty(base):
    
    __tablename__ = 'irysy'
    
    kwiat_id   = Column(Integer, primary_key=True)
    dzial_dlug = Column(Integer)
    dzial_szer = Column(Integer)
    plat_dlug  = Column(Integer)
    plat_szer  = Column(Integer)
    
    def __init__(self, kwiat_id, dzial_dlug, dzial_szer, plat_dlug, plat_szer):
        self.kwiat_id = kwiat_id
        self.dzial_dlug = dzial_dlug
        self.dzial_szer = dzial_szer
        self.plat_dlug = plat_dlug
        self.plat_szer = plat_szer

In [ ]:
# tworzenie tabel
base.metadata.create_all(engine)

In [ ]:
from sqlalchemy.orm import sessionmaker
# Stworzenie nowej sesji
Session = sessionmaker(bind=engine)
session = Session()
# dodanie danych
for t in range(10):
    tr = Transakcje(t, f'200{t}/05/06', t**2-t*2, 19)
    session.add(tr)
# zapis zmian w bazie danych
session.commit()

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
# dodanie danych
for t in range(df.shape[0]):
    tr = Kwiaty(t, df.iloc[t,0], df.iloc[t,1], df.iloc[t,2],df.iloc[t,3])
    session.add(tr)
# zapis zmian w bazie danych
session.commit()

In [ ]:
engine.execute("SELECT * FROM irysy").fetchall()

In [ ]:
engine.execute("SELECT * FROM Transakcje").fetchall()

In [ ]:
engine.execute("SELECT * FROM transakcje").fetchall()

In [ ]:
# wszystkie dane
for s in session.query(Transakcje).all():
    print(s.transakcja_id, s.data)

In [ ]:
# wybrane transakcje
for s in session.query(Transakcje).filter(Transakcje.transakcja_id>5):
    print(s.transakcja_id, s.data)

In [ ]:
# wybrane Kwiaty
for s in session.query(Kwiaty).filter(Kwiaty.dzial_dlug>5):
    print(s.kwiat_id, s.dzial_dlug)

In [ ]:
import json

JSON - Java Script Object Notation (ale w żaden sposób nie jest połączony z Java Scriptem czy żadnym innym językiem).  
JSON to sposób na lekki format wymiany danych.  
W JSON są czyste dane, nie ma tam jakichś zmiennych czy fukncji. Popularyazcja jego nastąpiła od mniej więcej 2001. Teraz jest używanie prawie, że domyślnie do wymiany danych między aplikacjami.

In [ ]:
from IPython.display import HTML
from IPython.display import Image

In [ ]:
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/haYYypSnOTY" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

In [ ]:
person = '{"name": "Bob", "languages": ["English", "French"]}'
type(person)

In [ ]:
person

In [ ]:
person_dict = json.loads(person)

# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
print(person_dict)

# Output: ['English', 'French']
print(person_dict['languages'])

In [ ]:
person_dict

In [ ]:
type(person_dict)

In [ ]:
%%file test5.json


{"name": "Bob", 
"languages": ["English", "French"]
}

In [ ]:
with open('test5.json') as f:
    data = json.load(f)

# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
print(data)

In [ ]:
type(data)

In [ ]:
data['name']

In [ ]:
data['languages']

In [ ]:
person_dict = {'name': 'Bob',
'age': 12,
'children': None
}

person_json = json.dumps(person_dict)
print(person_json)

In [ ]:
with open('person2.json', 'w') as json_file:
    json.dump(person_dict, json_file)

In [ ]:
print(json.dumps(person_dict, indent = 4, sort_keys=True))

In [ ]:
!pip install flask
!pip install flask_restful
!pip install flask_jsonpify

In [ ]:
from flask import Flask, request
from flask_restful import Resource, Api
from flask_jsonpify import jsonify

In [ ]:
app  = Flask(__name__)
api = Api(app)

@app.route('/')
@app.route('/index')
def home():
    #return render_template('home.html')
    return "Strona poczatkowa"

@app.route('/hello/<name>')
def success(name):
    return f'<h1>{name}</h1>'

In [ ]:
class Main(Resource):
    def get(self):
        return jsonify("Hello world")

In [ ]:
api.add_resource(Main,'/test')

In [ ]:
class Irys(Resource):
    
    def get(self):
        conn=engine.connect()
        query = conn.execute('select * from irysy')
        result = {'dane':[i for i in query.cursor.fetchall()]}
        return jsonify(result)

In [ ]:
api.add_resource(Irys, '/irys')

In [ ]:
app.run(port='5002')